In [100]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-top-1000-movies/imdb_clean.csv
/kaggle/input/imdb-top-1000-movies/imdb_clean2.csv


## Data Preparation

In [101]:
df = pd.read_csv("/kaggle/input/imdb-top-1000-movies/imdb_clean.csv")
df.head(5)

,title,release_year,runtime,genre,rating,gross(M)
0,The Shawshank Redemption,1994,142 min,Drama,9.3,28.34
1,The Godfather,1972,175 min,Crime,9.2,134.97
2,The Godfather,1972,175 min,Drama,9.2,134.97
3,The Dark Knight,2008,152 min,Action,9.0,534.86
4,The Dark Knight,2008,152 min,Crime,9.0,534.86


In [102]:
## Cleaning the runtime column values.

df["runtime"] = df["runtime"].apply(lambda x : x.replace("min",""))
df["runtime"] = df["runtime"].astype(int)
df.head(5)

,title,release_year,runtime,genre,rating,gross(M)
0,The Shawshank Redemption,1994,142,Drama,9.3,28.34
1,The Godfather,1972,175,Crime,9.2,134.97
2,The Godfather,1972,175,Drama,9.2,134.97
3,The Dark Knight,2008,152,Action,9.0,534.86
4,The Dark Knight,2008,152,Crime,9.0,534.86


In [103]:
## Renaming the runtime column

df = df.rename(columns={
    "runtime":"runtime_min"
})

## Renaming the gross(M) column

df = df.rename(columns={
    "gross(M)":"gross_usd_mil"
})

df.head(5)

,title,release_year,runtime_min,genre,rating,gross_usd_mil
0,The Shawshank Redemption,1994,142,Drama,9.3,28.34
1,The Godfather,1972,175,Crime,9.2,134.97
2,The Godfather,1972,175,Drama,9.2,134.97
3,The Dark Knight,2008,152,Action,9.0,534.86
4,The Dark Knight,2008,152,Crime,9.0,534.86


In [124]:
## combining genres in single rows.


## storing all titles in the title column:
myset = set()
for x in df["title"]:
    myset.add(x)
    
## creating a new dictionary for title + genre list
newdict = {}
for x in myset:
    just_genres = df.loc[df["title"] == x]["genre"]
    genre_list = []
    for g in just_genres:
        genre_list.append(g)
    newdict[x] = genre_list
    
## Creating a new df
newdf = pd.DataFrame()

## modifying the df
for x in newdict.keys():
    genre_string = ", ".join(newdict[x])
    df.loc[df["title"]==x,"genre"] = genre_string
    
df = df.drop_duplicates()
df.head(5)

,title,release_year,runtime_min,genre,rating,gross_usd_mil
0,The Shawshank Redemption,1994,142,Drama,9.3,28.34
1,The Godfather,1972,175,"Crime, Drama, Crime, Drama",9.2,134.97
3,The Dark Knight,2008,152,"Action, Crime, Drama, Action, Crime, Drama, Ac...",9.0,534.86
6,Schindler's List,1993,195,"Biography, Drama, History, Biography, Drama, H...",9.0,96.90
9,12 Angry Men,1957,96,"Crime, Drama, Crime, Drama",9.0,4.36
...,...,...,...,...,...,...
2518,Philomena,2013,98,"Biography, Comedy, Drama, Biography, Comedy, D...",7.6,37.71
2521,Control,2007,122,"Biography, Drama, Music, Biography, Drama, Mus...",7.6,0.87
2524,A Very Long Engagement,2004,133,"Drama, Mystery, Romance, Drama, Mystery, Roman...",7.6,6.17
2527,The Invisible Man,1933,71,"Horror, Sci-Fi, Horror, Sci-Fi",7.6,0.00


In [126]:
## Sorting the data frame from newest to oldest.

df = df.sort_values(by="release_year",ascending=False)
df

,title,release_year,runtime_min,genre,rating,gross_usd_mil
350,Guardians of the Galaxy Vol. 3,2023,150,"Action, Adventure, Comedy, Action, Adventure, ...",8.2,0.00
1177,John Wick: Chapter 4,2023,169,"Action, Crime, Thriller, Action, Crime, Thrill...",7.9,0.00
16,Spider-Man: Across the Spider-Verse,2023,140,"Animation, Action, Adventure, Animation, Actio...",8.9,0.00
1509,Everything Everywhere All at Once,2022,139,"Action, Adventure, Comedy, Action, Adventure, ...",7.8,72.86
1506,The Batman,2022,176,"Action, Crime, Drama, Action, Crime, Drama, Ac...",7.8,369.35
...,...,...,...,...,...,...
787,The Gold Rush,1925,95,"Adventure, Comedy, Drama, Adventure, Comedy, D...",8.1,5.45
517,Sherlock Jr.,1924,45,"Action, Comedy, Romance, Action, Comedy, Roman...",8.2,0.98
1315,Nosferatu,1922,94,"Fantasy, Horror, Fantasy, Horror",7.9,0.00
335,The Kid,1921,68,"Comedy, Drama, Family, Comedy, Drama, Family, ...",8.3,5.45


In [129]:
df.to_csv("movies_dataset.csv")